In [9]:
import pandas as pd

class EventLogger:
    def __init__(self):
        self.logs = []
    
    def log_customer_arrival(self, customer, time):
        self.logs.append({'event_time': time, 'event_name': 'customer_arrival', 'customer': customer})
    
    def log_customer_request(self, customer, time, flavor, quantity):
        self.logs.append({'event_time': time, 'event_name': 'customer_request', 'customer': customer, 'flavor': flavor, 'quantity_requested': quantity})

    def log_customer_served(self, customer, time, flavor, quantity):
        self.logs.append({'event_time': time, 'event_name': 'customer_served', 'customer': customer, 'flavor': flavor, 'quantity_served': quantity})

    def log_flavor_out_of_stock(self, customer, time, flavor):
        self.logs.append({'event_time': time, 'event_name': 'flavor_out_of_stock', 'customer': customer, 'flavor': flavor})
    
    def log_customer_unsatisfied(self, customer, time):
        self.logs.append({'event_time': time, 'event_name': 'customer_unsatisfied', 'customer': customer})
    
    def log_customer_departure(self, customer, time):
        self.logs.append({'event_time': time, 'event_name': 'customer_departure', 'customer': customer})

    def log_supplier_call(self, time, flavor, flavor_level):
        self.logs.append({'event_time': time, 'event_name': 'supplier_call', 'flavor': flavor, 'flavor_level': flavor_level})
    
    def log_supplier_arrival(self, time, flavor, flavor_level):
        self.logs.append({'event_time': time, 'event_name': 'supplier_arrival', 'flavor': flavor, 'flavor_level': flavor_level})

    def log_supplier_departure(self, time, flavor, flavor_level):
        self.logs.append({'event_time': time, 'event_name': 'supplier_departure', 'flavor': flavor, 'flavor_level': flavor_level})

    def get_logs_df(self):
        return pd.DataFrame(self.logs)
    
    def dump_logs_df(self, filepath=None):
        if filepath is None: 
            filepath = "logs.csv"

        self.get_logs_df().to_csv(filepath, index=False)


In [10]:
import simpy

class IceCreamShop:
    def __init__(self, env, n_employees, flavors, refilling_speed, supplier_time, logger):
        '''
        Initialize the ice cream shop with the following parameters:
        - env: the simpy environment
        - n_employees: the number of employees serving customers
        - flavors: a dictionary with flavors as keys and their initial quantities as values
        - refilling_speed: the speed at which a flavor can be refilled
        - supplier_time: the time it takes for the supplier to arrive
        - logger: the logger object to log events
        '''

        self.env = env
        self.employees = simpy.Resource(self.env, capacity=n_employees)
        self.flavors = {flavor: simpy.Container(self.env, capacity=quantity, init=quantity) for flavor, quantity in flavors.items()}

        self.refilling_speed = refilling_speed
        self.supplier_time = supplier_time
        self.supplier_called = {flavor: False for flavor in flavors}
        
        self.logger = logger


    def serve_customer(self, flavor_requested, quantity_requested):
        '''
        The serving process for a customer
        - flavor_requested: the flavor of ice cream the customer wants
        - quantity_requested: the amount of ice cream the customer wants
        '''

        yield self.flavors[flavor_requested].get(quantity_requested)
        # The "actual" serving process takes some time
        yield self.env.timeout(quantity_requested / self.refilling_speed)

    def refill_flavor(self, flavor):
        '''
        The process for refilling a specific flavor's tub
        - flavor: the flavor to be refilled
        '''
        self.supplier_called[flavor] = True
        
        yield self.env.timeout(self.supplier_time)
        self.logger.log_supplier_arrival(self.env.now, flavor, self.flavors[flavor].level)

        amount = self.flavors[flavor].capacity - self.flavors[flavor].level # top up the tub
        yield self.flavors[flavor].put(amount)
        
        print(f'{self.env.now:6.1f} s: Supplier arrived and refilled {flavor} with {amount:.1f} units')
        self.logger.log_supplier_departure(self.env.now, flavor, self.flavors[flavor].level)
        self.supplier_called[flavor] = False

    def flavor_control(self, flavor, threshold):
        '''
        The process that periodically checks the level of a flavor's tub and calls the supplier (refill_flavor process) if the level falls below a threshold
        - flavor: the flavor to be monitored
        - threshold: the minimum level that triggers the supplier to be called
        '''

        while True:
            flavor_level_ratio = 100 * (self.flavors[flavor].level / self.flavors[flavor].capacity)
            if (flavor_level_ratio < threshold) and (not self.supplier_called[flavor]):
                # We need to call the supplier now!
                print(f'{self.env.now:6.1f} s: Calling supplier for {flavor}')
                self.logger.log_supplier_call(self.env.now, flavor, self.flavors[flavor].level)
                
                # Wait for the supplier to arrive and refill the flavor
                yield self.env.process(self.refill_flavor(flavor))

            yield self.env.timeout(10) # Check every 10 seconds


In [11]:
import simpy
import random

class Customer:
    def __init__(self, env, name, flavor_preferences, ice_cream_shop, logger):
        '''
        Initialize the customer with the following parameters:
        - env: the simpy environment
        - name: the name of the customer
        - flavor_preferences: a list of tuples containing flavor names and the quantity requested
        - ice_cream_shop: the ice cream shop object
        - logger: the logger object to log events
        '''

        self.env = env
        self.name = name
        self.flavor_preferences = flavor_preferences

        self.logger = logger

        self.env.process(self.run(ice_cream_shop))

    def run(self, ice_cream_shop):
        '''
        The flow of the customer through the ice cream shop
        - ice_cream_shop: the ice cream shop object
        '''

        print(f'{self.env.now:6.1f} s: {self.name} arrived at ice cream shop')
        self.logger.log_customer_arrival(self.name, self.env.now)

        with ice_cream_shop.employees.request() as req:
            # Request an employee to serve
            yield req

            # Try to get the preferred flavor
            for flavor, quantity in self.flavor_preferences:
                if ice_cream_shop.flavors[flavor].level >= quantity:
                    print(f'{self.env.now:6.1f} s: {self.name} requests {quantity:.1f} units of {flavor}')
                    self.logger.log_customer_request(self.name, self.env.now, flavor, quantity)

                    yield self.env.process(ice_cream_shop.serve_customer(flavor, quantity))

                    print(f'{self.env.now:6.1f} s: {self.name} served with {quantity:.1f} units of {flavor}')
                    self.logger.log_customer_served(self.name, self.env.now, flavor, quantity)
                    break
                else:
                    print(f'{self.env.now:6.1f} s: {flavor} is out of stock for {self.name}')
                    self.logger.log_flavor_out_of_stock(self.name, self.env.now, flavor)

            else:
                # If none of the preferred flavors are available
                print(f'{self.env.now:6.1f} s: {self.name} could not be served with any preferred flavor')
                self.logger.log_customer_unsatisfied(self.name, self.env.now)

        print(f'{self.env.now:6.1f} s: {self.name} departs the ice cream shop')
        self.logger.log_customer_departure(self.name, self.env.now)


In [12]:
import simpy
import random

def customer_generator(env, t_inter, ice_cream_shop, flavor_preferences, logger):
    '''
    The process that generates customers arriving at the ice cream shop
    - t_inter: the interval between customer arrivals
    - ice_cream_shop: the ice cream shop object
    - flavor_preferences: a list of possible flavor preferences for customers
    - logger: the logger object to log events
    '''

    i = 0
    while True:
        yield env.timeout(random.randint(t_inter[0], t_inter[1]))  # Random interval between arrivals
        preferences = random.choice(flavor_preferences)  # Randomly choose a set of flavor preferences for the customer
        Customer(env, f'Customer {i}', preferences, ice_cream_shop, logger)
        i += 1


In [14]:
import simpy
import random
import pandas as pd

# Simulation parameters
RANDOM_SEED = 42           # Random seed
SIM_TIME = 1000            # Simulation time (seconds)

# Resource parameters
N_EMPLOYEES = 3            # Number of employees
FLAVORS = {                # Initial quantities of each flavor
    'vanilla': 20,
    'chocolate': 15,
    'strawberry': 10,
    'mint': 5
}
REFILLING_SPEED = 2        # Rate of refilling ice cream tubs (units/second)
SUPPLIER_TIME = 300        # Time it takes for the supplier to arrive (seconds)
THRESHOLD = 25             # Flavor minimum level (% of full capacity)

# Customer (entity) parameters
FLAVOR_PREFERENCES = [     # List of possible flavor preferences for customers
    [('vanilla', 2), ('chocolate', 1)],
    [('strawberry', 3)],
    [('vanilla', 1), ('strawberry', 2)],
    [('chocolate', 2), ('mint', 1)]
]

# Entity generation parameters
T_INTER = [30, 60]        # Interval between customer arrivals [min, max] (seconds)

# Setup and start the simulation
print('Ice Cream Shop Simulation')
print('Running Simulation...')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Define logger
logger = EventLogger()

# Define resources
ice_cream_shop = IceCreamShop(env, N_EMPLOYEES, FLAVORS, REFILLING_SPEED, SUPPLIER_TIME, logger)

# Define processes
for flavor in FLAVORS:
    env.process(ice_cream_shop.flavor_control(flavor, THRESHOLD))
env.process(customer_generator(env, T_INTER, ice_cream_shop, FLAVOR_PREFERENCES, logger))

# Execute
env.run(until=SIM_TIME)
print('... Done \n')

# Print logs or save them to a file
print(logger.get_logs_df())
logger.dump_logs_df("ice_cream_shop_logs.csv")


Ice Cream Shop Simulation
Running Simulation...
  50.0 s: Customer 0 arrived at ice cream shop
  50.0 s: Customer 0 requests 2.0 units of vanilla
  51.0 s: Customer 0 served with 2.0 units of vanilla
  51.0 s: Customer 0 departs the ice cream shop
  80.0 s: Customer 1 arrived at ice cream shop
  80.0 s: Customer 1 requests 1.0 units of vanilla
  80.5 s: Customer 1 served with 1.0 units of vanilla
  80.5 s: Customer 1 departs the ice cream shop
 117.0 s: Customer 2 arrived at ice cream shop
 117.0 s: Customer 2 requests 3.0 units of strawberry
 118.5 s: Customer 2 served with 3.0 units of strawberry
 118.5 s: Customer 2 departs the ice cream shop
 151.0 s: Customer 3 arrived at ice cream shop
 151.0 s: Customer 3 requests 2.0 units of vanilla
 152.0 s: Customer 3 served with 2.0 units of vanilla
 152.0 s: Customer 3 departs the ice cream shop
 202.0 s: Customer 4 arrived at ice cream shop
 202.0 s: Customer 4 requests 2.0 units of vanilla
 203.0 s: Customer 4 served with 2.0 units of va